In [1]:
# Cell 1: Imports
from markdown import markdown
from playwright.async_api import async_playwright
from pathlib import Path
import asyncio



In [2]:
# Cell 2: Async conversion function
async def md_to_pdf_async(md_file, pdf_file, wait_time=2000):
    """
    Convert markdown to PDF with full support for:
    - Mermaid diagrams
    - LaTeX math
    - Code blocks with syntax highlighting
    - Tables, images, links
    - Callouts/admonitions
    """
    # Read markdown
    with open(md_file, 'r', encoding='utf-8') as f:
        md_content = f.read()
    
    # Convert with extensions
    html_content = markdown(md_content, extensions=[
        'tables',
        'fenced_code',
        'codehilite',
        'toc',
        'attr_list',
        'md_in_html'
    ])
    
    # Full HTML template
    full_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/github-markdown-css/5.2.0/github-markdown.min.css">
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.8.0/styles/github.min.css">
        <script src="https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.min.js"></script>
        <script>
            MathJax = {{
                tex: {{ inlineMath: [['$', '$'], ['\\(', '\\)']] }},
                svg: {{ fontCache: 'global' }}
            }};
        </script>
        <script src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"></script>
        <style>
            .markdown-body {{
                box-sizing: border-box;
                min-width: 200px;
                max-width: 980px;
                margin: 0 auto;
                padding: 45px;
            }}
            .callout {{
                padding: 15px;
                margin: 15px 0;
                border-left: 4px solid #2196F3;
                background: #E3F2FD;
                border-radius: 4px;
            }}
            .callout-warning {{
                border-left-color: #FF9800;
                background: #FFF3E0;
            }}
            .callout-danger {{
                border-left-color: #F44336;
                background: #FFEBEE;
            }}
            @media print {{
                body {{ margin: 0; }}
                .markdown-body {{ padding: 20px; }}
            }}
        </style>
    </head>
    <body class="markdown-body">
        {html_content}
        <script>
            mermaid.initialize({{ startOnLoad: true, theme: 'default' }});
        </script>
    </body>
    </html>
    """
    
    # Convert to PDF
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.set_content(full_html, wait_until='networkidle')
        
        # Wait for JavaScript to render (mermaid, mathjax)
        await page.wait_for_timeout(wait_time)
        
        # Generate PDF
        await page.pdf(
            path=pdf_file,
            format='A4',
            print_background=True,
            margin={'top': '20mm', 'bottom': '20mm', 'left': '20mm', 'right': '20mm'}
        )
        await browser.close()
    
    print(f"✅ PDF created: {pdf_file}")



In [3]:
# Cell 3: Create test markdown (same as before)
test_md = """
# Test Document

## Features Test

### 1. Lists
- Bullet point 1
- Bullet point 2
  - Nested item
  - Another nested

1. Numbered item
2. Second item

### 2. Code Block
```python
def hello_world():
    print("Hello from code block!")
    return 42
```

### 3. Table
| Feature | Status |
|---------|--------|
| Tables  | ✅     |
| Math    | ✅     |
| Diagrams| ✅     |

### 4. LaTeX Math
Inline math: $E = mc^2$

Block math:
$$\\frac{-b \\pm \\sqrt{b^2-4ac}}{2a}$$

### 5. Mermaid Diagram
```mermaid
graph TD
    A[Start] --> B{Decision}
    B -->|Yes| C[Success]
    B -->|No| D[Retry]
    D --> B
```

### 6. Checkboxes
- [x] Completed task
- [ ] Pending task
- [ ] Another task

### 7. Links & Images
[GitHub](https://github.com)

**Bold text** and *italic text* and ~~strikethrough~~
"""

with open('test.md', 'w', encoding='utf-8') as f:
    f.write(test_md)

print("✅ Test markdown created")



✅ Test markdown created


In [4]:
# Cell 4: Run the conversion (note the await)
await md_to_pdf_async('test.md', 'test.pdf', wait_time=3000)



✅ PDF created: test.pdf


In [5]:
# Cell 5: Check result
import os
if os.path.exists('test.pdf'):
    print(f"✅ Success! PDF size: {os.path.getsize('test.pdf')} bytes")
else:
    print("❌ PDF not created")

✅ Success! PDF size: 145478 bytes
